In [76]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf


## sorotan:

In [119]:
df = pd.read_csv("https://drive.google.com/uc?id=1AUG66tl_Up3EX-dGssA19anKBgcteBBK")
df.Headline

df0 = df[df.Label==0].copy().reset_index(drop=True)
df1 = df[df.Label==1].copy().reset_index(drop=True)

df.Label.value_counts(), df.shape

(Label
 0    683
 1    433
 Name: count, dtype: int64,
 (1116, 3))

In [78]:
# %reload_ext google.colab.data_table

df.head(3)

# 37k unique words (not really but yeah)
print(df.Body.str.lower().apply(lambda x: re.split('[ \.,?!#\(\)=:]', x)).explode().nunique())

# uh, yeah?
df.Body.str.lower().apply(lambda x: re.split('[ \.,?!#\(\)=:]', x)).explode().value_counts()[:100].index

36825


Index(['', 'yang', 'di', 'dan', 'tidak', 'itu', 'dari', 'tersebut', 'ini',
       'dengan', '”', 'untuk', ' ', 'ada', 'dalam', 'pada', 'adalah', 'oleh',
       'bahwa', 'ke', 'media', 'sumber', 'juga', 'akan', 'sudah', 'indonesia',
       'atau', 'the', 'karena', 'saya', 'foto', 'hoax', 'saat', '1', 'sosial',
       'bisa', 'informasi', 'telah', 'mereka', 'kepada', 'klarifikasi',
       'referensi', 'penjelasan', 'kami', 'narasi', 'akun', 'orang', '2',
       'sebagai', 'seperti', 'tahun', 'berita', 'mengatakan', 'kata',
       'masyarakat', 'pihak', 'menjadi', 'satu', 'kita', 'beredar', 'jika',
       'com', 'yg', 'hanya', 'of', 'bukan', 'benar', 'atas', 'melalui', 'dia',
       '–', 'lebih', 'hal', 'jakarta', 'beberapa', 'salah', '3', 'melakukan',
       'pesan', 'video', 'pun', 'and', 'dapat', 'to', 'kabar', 'pernah',
       'namun', 'warga', 'presiden', 'sebuah', 'terkait', 'surat', 'hari', 'a',
       'terjadi', 'belum', 'tak', 'lalu', 'in', 'ia'],
      dtype='object', name='Body

In [79]:
# df.Body.str.split("=======", expand=True)

In [80]:
# df.loc[df.Label==1, "Body"].str.split("=====", expand=True)

In [81]:
# df.loc[df.Label==1, "Body"].str.split(" ").explode().value_counts()[:100].index()


In [134]:
titlecase = df.loc[df.Label==0, "Body"].apply(lambda w: re.findall('([A-Z][a-z]+)', w))
print(titlecase.explode().value_counts()[:30])

uppercase = df.loc[df.Label==0, "Body"].apply(lambda w: re.findall('([A-Z][A-Z]+)', w))
(uppercase.explode().value_counts()[:30])

Body
Indonesia     757
Sumber        526
Jakarta       332
Narasi        256
Facebook      255
Media         255
Anda          231
The           231
Dalam         219
Sosial        213
Penjelasan    211
Islam         207
Presiden      201
Dan           196
Foto          191
Saya          183
Namun         182
Hoax          180
Ahok          176
Di            163
Tidak         162
Jadi          159
China         158
Berikut       155
Fakta         152
Referensi     148
Ini           140
Dengan        138
Jokowi        137
Menurut       136
Name: count, dtype: int64


Body
SUMBER        388
PENJELASAN    357
NARASI        319
REFERENSI     296
TNI           201
PT            151
DKI           138
HOAX          120
FAFHH         104
PKI            99
MEDIA          94
RI             75
SOSIAL         73
LINK           71
FPI            60
DI             60
WIB            57
TIDAK          52
BPOM           52
TV             51
MUI            48
AS             46
KPU            45
UU             44
AKBP           43
HIV            43
KH             42
DAN            41
CATATAN        39
KPK            38
Name: count, dtype: int64

Index(['SUMBER', 'PENJELASAN', 'NARASI', 'REFERENSI', 'TNI', 'PT', 'DKI',
       'HOAX', 'FAFHH', 'PKI', 'MEDIA', 'RI', 'SOSIAL', 'LINK', 'FPI', 'DI',
       'WIB', 'TIDAK', 'BPOM', 'TV', 'MUI', 'AS', 'KPU', 'UU', 'AKBP', 'HIV',
       'KH', 'DAN', 'CATATAN', 'KPK', 'SMS', 'PDIP', 'UI', 'HT', 'BPPKB',
       'HTI', 'INI', 'AQUA', 'DPR', 'WA', 'BBM', 'SIM', 'LGBT', 'POM',
       'WHATSAPP', 'PP', 'US', 'RS', 'AOCT', 'RW', 'KTP', 'CO', 'ISIS',
       'JANGAN', 'BRI', 'UNTUK', 'NKRI', 'CNN', 'AIDS', 'CCTV', 'YANG', 'PLN',
       'BUKAN', 'BI', 'FAKTANYA', 'PTUN', 'INDONESIA', 'HAM', 'KESIMPULAN',
       'KAMI', 'HP', 'KK', 'MIE', 'YG', 'GO', 'DISINFORMASI', 'NU', 'PBB',
       'DENGAN', 'SEMUA', 'ITE', 'AKP', 'FAKTA', 'SMA', 'PNS', 'UGM', 'HPV',
       'BPJS', 'TPG', 'AHOK', 'KANKER', 'JNE', 'KLAIM', 'JAKARTA', 'SINDO',
       'ISLAM', 'BNN', 'ORANG', 'HATI', 'II'],
      dtype='object', name='Body')

## Label One

### Label 1 unique words (?)
1. titlecase: klarifikasi, sumber, kategori, referensi, dalam, media, twitter, facebook, penjelasan, dilansir, narasi, menurut, menanggapi, dari, beredar
2. uppercase: referensi, penjelasan, sumber, klarifikasi, kategori, narasi, media, daring, hoax, sosial

In [85]:
%%capture cell
titlecase = df1["Body"].apply(lambda w: re.findall('([A-Z][a-z]+)', w))
uppercase = df1["Body"].apply(lambda w: re.findall('([A-Z][A-Z]+)', w))


print(titlecase.explode().value_counts()[:100].index)
print(uppercase.explode().value_counts()[:100].index)

In [91]:
# split by klarifikasi --> can't do
df1k = df1["Body"].str.split("KLARIFIKASI", expand=True)
# display(df1k.head(2))

# so less than half got klarifikasi in their so yeah.
print(df1k.shape), print(df1k.isna().sum())
df1k.dropna()

(433, 3)
0      0
1    294
2    431
dtype: int64


,0,1,2
216,“Foto-foto yang beredar mengenai Orang² yang s...,Foto-foto yang beredar mengenai Orang² yang s...,:* *Kronologi Kejadian :* Kita harus mulai da...
316,"“Perlu kita tahu saudara-saudaraku, bahwa Kapo...",====== Sumber : Media Sosial Facebook =...,VIDEO KAPOLRES LANDAK AKBP BOWO GEDE IMANTIO ...


In [87]:
def joiner(x):
  return "===".join([k.strip() for k in x if len(k.split()) > 10])


In [88]:
# split by klarifikasi --> can't do
df1l = df1["Body"].str.split("=====")
display(df1l.head(2))

# so less than half got klarifikasi in their so yeah.
print(df1l.shape), print(df1l.isna().sum())

# let's remove some short sentences < 10 and return the rest

df1l = df1["Body"].str.split("=====").apply(joiner).str.split("===").apply(joiner).str.split("===", expand=True)

print(df1l.shape, df1l.isna().sum())

0    [Foto: Kumparan.com Debunk ini berisi iKlarifi...
1    [Foto Ilustrasi Perkosaan Debunk ini berisi kl...
Name: Body, dtype: object

(433,)
0
(433, 6) 0      0
1      2
2    170
3    377
4    417
5    430
dtype: int64


In [96]:
display(df1[["Headline"]].head(), df0[["Headline"]].head())

,Headline
0,Pertamina dan Pemerintah Kalrifikasi Isu BBM N...
1,Polda Nusa Tenggara Barat Mengklarifkasi Bahwa...
2,PT Ultrajaya Milk Industry Tbk Mengklarifikasi...
3,BBKSDA Jatim Mengklarifikasi Bahwa Wisata Alam...
4,BMKG Mataram Menghimbau Agar Warga Tidak Langs...


,Headline
0,“Cacing Hidup di Sarden” | “Living Worm in Sa...
1,“Para Bajingan Tengik PKI Sudah Mulai Terang-t...
2,“Terjadi Lagi Pembunuhan Terhadap Muadzin”
3,Emil Salim Meninggal
4,Pengeroyokan Santri di Garut


## Label 0

### Label 0 unique words (?)
1. titlecase: sumber, narasi, facebook, media, penjelasan, hoax, fakta, referensi, menurut, foto, google, whatsapp, berdasarkan, berita, pesan, informasi, kabar, app
2. uppercase: sumber, penjelasan, narasi, referensi, hoax, media, catatan, whatsapp, faktanya, kesimpulan, disinformasi, fakta, klaim,


In [ ]:
def joiner2(x):
  x = x.split("NARASI")
  x = "===".join([k.strip() for k in x if len(k.split()) > 10])
  x =


In [128]:

# split by klarifikasi --> can't do
df0l = df0["Body"].str.split("=====")
display(df0l.head(2))

# so less than half got klarifikasi in their so yeah.
print(df0l.shape), print(df0l.isna().sum())

# let's remove some short sentences < 10 and return the rest

df0l = df0["Body"].str.split("NARASI|PENJELASAN|KESIMPULAN|Narasi|Penjelasan|Kesimpulan", expand=True)

print(df0l.shape, df0l.isna().sum())

0    [Cacing tidak akan bertahan hidup melalui pros...
1    [Sudah ditangani pihak yang berwenang: “Bandun...
Name: Body, dtype: object

(683,)
0
(683, 47) 0       0
1     124
2     190
3     618
4     657
5     674
6     678
7     679
8     680
9     680
10    681
11    681
12    681
13    681
14    682
15    682
16    682
17    682
18    682
19    682
20    682
21    682
22    682
23    682
24    682
25    682
26    682
27    682
28    682
29    682
30    682
31    682
32    682
33    682
34    682
35    682
36    682
37    682
38    682
39    682
40    682
41    682
42    682
43    682
44    682
45    682
46    682
dtype: int64


In [139]:
df0.Body[3]

'Beredar kabar di media pesan yang menyatakan Emil Salim, Guru Besar Fakultas Ekonomi Universitas Indonesia dan Mantan Menteri Negara Pengawasan Pembangunan dan Lingkungan Hidup di Era Presiden Soeharto, telah meninggal dunia. Dilansir dari\xa0detik.com\xa0(4/3), Ari Kuncoro, Dekan Fakultas Ekonomi dan Bisnis Universitas Indonesia (UI), mengatakan, kabar tersebut adalah hoaks. “Ya saya dapat informasi, dan saya katakan hal tersebut hoax. Saya dapat informasi dari UI, itu hoax,” ujar Ari. ===== Kategori: Hoaks ===== Sumber: Whatsapp ===== Narasi: Aslkm wr wb Innanilahi wa innaillahi rojiun Selamat Jalan Papa Bpk Emil Muhammad bin Salim bin Muhamad Djawas Jam 02.00 dini hari Minggu, 4 Maret 2018 ===== Penjelasan Lengkap: Pada Minggu, 4 Maret 2018, beredar kabar melalui media pesan Whatsapp yang menyatakan Emil Salim, Guru Besar Fakultas Ekonomi Universitas Indonesia, telah meninggal dunia. Pria yang juga pernah menjabat sebagai Menteri Negara Pengawasan Pembangunan dan Lingkungan Hidup d